In [1]:
%matplotlib qt
import scipy
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import osascript
from gtts import gTTS 
import os 
import pyaudio
import wave
import keyboard  # using module keyboard
import soundfile as sf
import math
import pyloudnorm as pyln
from sys import byteorder
from array import array
from struct import pack
import librosa
from scipy.signal import butter, sosfiltfilt
import pysptk
import python_speech_features
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
import time
from sklearn.model_selection import KFold
from keras.optimizers import Adadelta
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import keras as keras
from  conch.analysis.formants import lpc
import sklearn
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras import backend as K

Using TensorFlow backend.


In [2]:
X_train, X_val, X_test, y_train, y_val, y_test = np.load("../../splitdata.npy", allow_pickle=True)

In [23]:
dataset, fs, y = np.load("../../CREMA/CREMAdatafsy.npy", allow_pickle=True)

In [32]:
chunk = int(512*(fs/1000))
one_point = dataset[0][0:chunk]
one_point.shape, fs, chunk

((8192,), 16000, 8192)

In [ ]:
def remove_silence_from(amplitudes, threshold):
    silenced = []
    for x in amplitudes:
        if x >= threshold:
            silenced.append(x)
    return silenced

In [4]:
def MFCC_algorithm(np_data, fs):
    MFCCs = []
    print("running .....")
    #for progess bar
    for one_sound in np_data:
        
        one_sound = np.asarray(one_sound).reshape(int((fs/1000)*512), 1)
        
        MFCCs.append(python_speech_features.base.mfcc(one_sound, samplerate=fs, 
                                     winlen=0.025, winstep=0.01, numcep=13, 
                                     nfilt=26, nfft=552))
    return np.array(MFCCs)

In [118]:
def get_pitch_vector(data, fs):
    data = np.float32(data)
    pitch = pysptk.sptk.rapt(y=data, sr=fs, hopsize = 40)
    silenced = remove_silence_from(pitch, np.mean(pitch))
    return silenced

In [154]:
def get_spectral_vector(data, fs):
    data = np.float32(data)
    cent = librosa.feature.spectral_centroid(y=data, sr=fs, hop_length=165)
    return cent

In [177]:
def get_rms_vector(data):
    temp_data = np.float32(data)
    cent = librosa.feature.rms(y=temp_data, hop_length=165)
    return cent

In [183]:
def get_zero_vector(data):
    temp_data = np.float32(data)
    cent = librosa.feature.zero_crossing_rate(y=temp_data, hop_length=165)
    return cent

In [187]:
def get_sr_vector(data):
    temp_data = np.float32(data)
    cent = librosa.feature.spectral_rolloff(y=temp_data, hop_length=165)
    return cent

In [196]:
def me_and_gradient(x):
    return x, np.gradient(x)

In [ ]:
print(len((get_spectral_vector(one_point, fs))[0]))

print(len(get_spectral_vector(one_point, fs)[0]))

print(len(get_zero_vector(one_point)[0]))

print(len(get_rms_vector(one_point)[0]))

print(len(get_sr_vector(one_point)[0]))


In [2]:
fs = 16000
dataset, y, useless_number = np.load("../../CREMA_chunked.npy", allow_pickle=True)

In [5]:
X = MFCC_algorithm(dataset, fs)
X = X.reshape(X.shape[0], 50, 13, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

running .....


In [6]:
print("x train shape: " +str(X_train.shape))
print("y train shape: " +str(y_train.shape))
print("x test shape: " +str(X_test.shape))
print("y test shape: " +str(y_test.shape))

x train shape: (32499, 50, 13, 1)
y train shape: (32499, 12)
x test shape: (4063, 50, 13, 1)
y test shape: (4063, 12)


In [7]:
def cnn_MFCC_LSTM(X_train, X_test, y_train, y_test):
    # Set the target class number
    
    # VGG 19
    
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    
    
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu', input_shape = (50,13,1)))
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    print(model.input_shape)
    print(model.output_shape)
    
    model.add(Conv2D(128, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    print(model.output_shape)
    
    model.add(Conv2D(256, kernel_size = (3,3), activation='relu'))
    model.add(Conv2D(256, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    print(model.output_shape)
    
    
    model.add(Conv2D(512, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    print(model.output_shape)
    
    
    
    model.add(Flatten())
    print("flattened: ")
    print(model.output_shape)
    
    
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    print(model.output_shape)
    
    model.add(Dense(target_class, activation='softmax'))
    print(model.output_shape)
    
    
    
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=0.0, amsgrad=False)
    
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,metrics=[keras.metrics.categorical_accuracy])
    cnnhistory = model.fit(X_train, y_train, batch_size=X_train.shape[0], epochs=40, 
                         validation_data=(X_test, y_test))
    
    
    a = model.predict(X_train)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(predictions)
    print(y_train)
    print(classification_report(y_train,predictions))
    a = model.predict(X_test)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(classification_report(y_test,predictions))
    
    
    

    return model

In [ ]:
cnn_MFCC_LSTM(X_train, X_val, y_train, y_val)

W0726 18:34:22.253114 140735522395008 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 18:34:22.305541 140735522395008 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 18:34:22.310031 140735522395008 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 18:34:22.413871 140735522395008 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



(None, 50, 13, 1)
(None, 47, 10, 64)
(None, 46, 9, 128)
(None, 41, 4, 256)
(None, 19, 1, 512)
flattened: 
(None, 9728)
(None, 4096)
(None, 12)


W0726 18:34:23.361099 140735522395008 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 18:34:23.404896 140735522395008 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0726 18:34:23.852771 140735522395008 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0726 18:34:24.455065 140735522395008 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Train on 32499 samples, validate on 4062 samples
Epoch 1/40


In [ ]:
parameters = [sherpa.Continuous(name='lr', range=[0.0001, 0.1], scale='log'),
              sherpa.Continuous(name='beta1', range=[0.85, 1.0], scale='log'),
              sherpa.Continuous(name="regularization", range=[0.0001, 1], scale='log'),
              sherpa.Continuous(name='dropout', range=[0.0, 0.5]),
              sherpa.Ordinal(name='batch_size', range=[16, 32, 64,128,256,512,1024, X_train.shape[0]]),
              sherpa.Choice(name='activation', range=['relu', 'elu', 'prelu', 'tanh'])]

alg =bayesian_optimization.GPyOpt()

study = sherpa.Study(parameters=parameters,
                     algorithm=alg,dashboard_port=9998,disable_dashboard=False,
                     lower_is_better=False)

for trial in study:
    cnn_MFCC_LSTM(study, trial, X_train, X_val, y_train, y_val, regularization =trial.parameters['regularization'],
              drop_likely = trial.parameters['dropout'],
              learning_rate = trial.parameters['lr'],
              beta1 = trial.parameters['beta1'], beta2 = 0.999,
             num_iterations = 150, size_batch=trial.parameters["batch_size"])
    study.finalize(trial)


In [198]:
def cnn_MFCC_LSTM(study, trial, X_train, X_test, y_train, y_test, regularization = 0.1,drop_likely = 0.1,
              learning_rate = 0.001, beta1 = 0.9, beta2 = 0.999,
             num_iterations = 40, size_batch=X_train.shape[0],activation = 'tanh'):
    # Set the target class number
    
    # VGG 19
    
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    
    
    model.add(Conv2D(64, kernel_size = (3,3), activation=activation, input_shape = (50,13,1)))
    model.add(Conv2D(64, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))

    
    model.add(Conv2D(128, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(128, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))

    
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))

    
    
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    
    
    
    model.add(Flatten())

    
    
    model.add(Dense(4096, activation=activation))
    model.add(Dense(4096, activation=activation))

    model.add(Dense(target_class, activation='softmax'))

    
    
    es = keras.callbacks.EarlyStopping(monitor='val_acc', mode='max',  patience=10, min_delta=0)
    
    opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=10e-8)
       
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,metrics=[keras.metrics.categorical_accuracy])
    
    nn_history = model.fit(X_train, y_train, batch_size=size_batch, epochs=num_iterations, 
                         validation_data=(X_test, y_test),
                          callbacks=[es, study.keras_callback(trial, objective_name='val_acc')])


    return model


In [101]:
def cnn_inception(X_train, X_test, y_train, y_test):
    #inception architecture
    
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    model.add(keras.layers. conv2d_bn(32, 3,3, stridea =(2,2)))
    

In [102]:
cnn_inception(X_cache, X_cache, Y_cache, Y_cache)

NameError: name 'Conv2D_bn' is not defined

In [3]:
Y_train = np.argmax(y_train, axis = 1)
Y_val = np.argmax(y_val, axis = 1)

In [12]:
model = sklearn.svm.NuSVC(gamma = 'auto' )
model.fit(X_train, Y_train)
predictions = model.predict(X_val)
print(classification_report(Y_val, predictions))


              precision    recall  f1-score   support

           0       0.46      0.62      0.53        73
           1       0.48      0.36      0.41        86
           2       0.46      0.36      0.41        77
           3       0.38      0.39      0.39        79
           4       0.45      0.48      0.46        69
           5       0.44      0.53      0.48        70
           6       0.65      0.76      0.70       110
           7       0.58      0.51      0.54       111
           8       0.70      0.45      0.55       110
           9       0.62      0.64      0.63        99
          10       0.65      0.68      0.66       107
          11       0.62      0.70      0.66       113

    accuracy                           0.55      1104
   macro avg       0.54      0.54      0.53      1104
weighted avg       0.56      0.55      0.55      1104



In [8]:
model = sklearn.svm.SVC(gamma = 'scale')
model.fit(X_train, Y_train)
predictions = model.predict(X_val)
print(classification_report(Y_val, predictions))

              precision    recall  f1-score   support

           0       0.51      0.63      0.56        73
           1       0.49      0.36      0.42        86
           2       0.48      0.32      0.39        77
           3       0.43      0.47      0.45        79
           4       0.44      0.52      0.48        69
           5       0.44      0.53      0.48        70
           6       0.71      0.79      0.75       110
           7       0.61      0.55      0.58       111
           8       0.66      0.52      0.58       110
           9       0.64      0.64      0.64        99
          10       0.65      0.72      0.68       107
          11       0.63      0.67      0.65       113

    accuracy                           0.57      1104
   macro avg       0.56      0.56      0.55      1104
weighted avg       0.57      0.57      0.57      1104



In [10]:
model = sklearn.linear_model.SGDClassifier()
model.fit(X_train, Y_train)
predictions = model.predict(X_val)
print(classification_report(Y_val, predictions))

              precision    recall  f1-score   support

           0       0.55      0.56      0.55        73
           1       0.35      0.30      0.32        86
           2       0.37      0.31      0.34        77
           3       0.24      0.25      0.25        79
           4       0.45      0.48      0.46        69
           5       0.41      0.29      0.34        70
           6       0.68      0.71      0.70       110
           7       0.56      0.45      0.50       111
           8       0.54      0.37      0.44       110
           9       0.58      0.41      0.48        99
          10       0.51      0.64      0.57       107
          11       0.39      0.68      0.49       113

    accuracy                           0.47      1104
   macro avg       0.47      0.46      0.45      1104
weighted avg       0.48      0.47      0.47      1104



In [79]:
regularization = 0.1
drop_likely = 0.1
learning_rate = 0.001
beta1 = 0.9 
beta2 = 0.999,
num_iterations = 40
size_batch=256,
activation = 'relu'
target_class = y_train.shape[1]
# Model ALEX NEt
model = Sequential()

model.add(Conv1D(96, 22, padding='valid',strides=2,input_shape=(X_train.shape[1], 1))) 
print(model.input_shape)
print(model.output_shape)
model.add(MaxPooling1D(pool_size = (3), strides=(2)))
print(model.output_shape)
model.add(Conv1D(256, 5, padding='same')) 

print(model.output_shape)
model.add(MaxPooling1D(pool_size = (3), strides=(2)))
print(model.output_shape)

model.add(Conv1D(384, 3, padding='same')) 
print(model.output_shape)

model.add(Conv1D(384, 3, padding='same')) 
print(model.output_shape)

model.add(Conv1D(256, 3, padding='same')) 
print(model.output_shape)

model.add(MaxPooling1D(pool_size = (3), strides=(2)))
print(model.output_shape)
model.add(Flatten())

model.add(Dense(4096, input_dim=9216,  activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))
model.add(keras.layers.BatchNormalization())

model.add(Dense(4096, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))
model.add(Dropout(drop_likely))

model.add(Dense(2048, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))
model.add(keras.layers.BatchNormalization())
model.add(Dropout(drop_likely))

model.add(Dense(1024, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))

model.add(Dense(512, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))

model.add(Dense(target_class, activation='softmax'))

es = keras.callbacks.EarlyStopping(monitor='val_acc', mode='max',  patience=10, min_delta=0)

opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=10e-8)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
nn_history = model.fit(X_train, y_train, batch_size=size_batch, epochs=num_iterations, 
                     validation_data=(X_val, y_val),callbacks=[es])
model, nn_history

(None, 131, 1)
(None, 55, 96)
(None, 27, 96)
(None, 27, 256)
(None, 13, 256)
(None, 13, 384)
(None, 13, 384)
(None, 13, 256)
(None, 6, 256)


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 8829 arrays: [array([[ 0.60435303,  1.65671061,  1.22716584, -0.28892201,  1.54634199,
         0.65472625, -1.4454241 , -0.46602093, -3.23933811, -0.96958441,
         1.89615119, -0.25373911, -0.43849836,  0.621...

In [ ]:
def complex_cnn(X_train, X_test, y_train, y_test):
    # Set the target class number
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    model.add(keras.layers.BatchNormalization())
    model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1], 1))) #1
    model.add(Activation('tanh'))
    model.add(Conv1D(256, 8, padding='same')) #2
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.25))
    model.add(Conv1D(128, 8, padding='same')) #3
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh')) 
    model.add(Conv1D(128, 8, padding='same')) #4
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Conv1D(128, 8, padding='same')) #5
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Conv1D(128, 8, padding='same')) #6
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.25))
    model.add(Conv1D(64, 8, padding='same')) #7
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Conv1D(64, 8, padding='same')) #8
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Flatten())
    model.add(Dense(target_class)) #9
    model.add(Activation('softmax'))
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,metrics=[keras.metrics.categorical_accuracy])
    cnnhistory = model.fit(X_train, y_train, batch_size=128, epochs=40, 
                         validation_data=(X_test, y_test))
    a = model.predict(X_train)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(predictions)
    print(y_train)
    print(classification_report(y_train,predictions))
    a = model.predict(X_test)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(classification_report(y_test,predictions))
    return model, cnnhistory